## Flopy data types, head and budget files

In order to access the binary data outputs created by MODFLOW, flopy uses [utility modules](https://modflowpy.github.io/flopydoc/binaryfile.html).

The class *flopy.utils.binaryfile.HeadFile* creates a Headfile object with mutliple methods.

In [2]:
import flopy
# read in the heads
headobj = flopy.utils.binaryfile.HeadFile('ET_Model_CheckET2.hds')

headobj.list_records()

(1, 1, 10., 10., b'            HEAD', 50, 50, 1)
(1, 2, 5., 15., b'            HEAD', 50, 50, 1)
(2, 2, 10., 20., b'            HEAD', 50, 50, 1)
(3, 2, 15., 25., b'            HEAD', 50, 50, 1)
(4, 2, 20., 30., b'            HEAD', 50, 50, 1)
(5, 2, 25., 35., b'            HEAD', 50, 50, 1)
(6, 2, 30., 40., b'            HEAD', 50, 50, 1)
(1, 3, 5., 45., b'            HEAD', 50, 50, 1)
(2, 3, 10., 50., b'            HEAD', 50, 50, 1)
(3, 3, 15., 55., b'            HEAD', 50, 50, 1)
(4, 3, 20., 60., b'            HEAD', 50, 50, 1)
(5, 3, 25., 65., b'            HEAD', 50, 50, 1)
(6, 3, 30., 70., b'            HEAD', 50, 50, 1)
(7, 3, 35., 75., b'            HEAD', 50, 50, 1)
(8, 3, 40., 80., b'            HEAD', 50, 50, 1)
(9, 3, 45., 85., b'            HEAD', 50, 50, 1)
(10, 3, 50., 90., b'            HEAD', 50, 50, 1)
(11, 3, 55., 95., b'            HEAD', 50, 50, 1)
(12, 3, 60., 100., b'            HEAD', 50, 50, 1)
(1, 4, 5., 105., b'            HEAD', 50, 50, 1)
(2, 4, 10., 110., 